In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
os.chdir('/Users/connorvoglewede/git-folder/boba')

In [77]:
import pandas as pd
import logging
import numpy as np
from bs4 import BeautifulSoup
import requests
import io
import re
from lxml import html 
from tqdm import tqdm
import json
import warnings 
import yaml
import boba.preprocessing as pp
warnings.filterwarnings("ignore")


In [261]:
pd.set_option('max_rows',50)
pd.set_option('max_columns',400)

In [313]:
season = 2021

per_metric_h = 'PA' 
pt_metric_h = 'PA'      
counting_stats_h = ['HR','R','RBI','WAR','SB','CS']
rate_stats_h = ['AVG','OBP','SLG','BABIP','BB%','K%','wOBA']
model_targets_h = rate_stats_h + [c+'_per_'+per_metric_h for c in counting_stats_h]  

per_metric_sp = 'GS' 
pt_metric_sp = 'IP'      
counting_stats_sp = ['ShO','CG','W','WAR']
rate_stats_sp = ['ERA','BB_per_9','K_per_9','OBP','SLG']
model_targets_sp = rate_stats_sp + [c+'_per_'+per_metric_sp for c in counting_stats_sp]  

per_metric_rp = 'G' 
pt_metric_rp = 'IP'      
counting_stats_rp = ['SV','HLD','WAR']
rate_stats_rp = ['ERA','BB_per_9','K_per_9','OBP','SLG']
model_targets_rp = rate_stats_rp + [c+'_per_'+per_metric_rp for c in counting_stats_rp] 


In [314]:
id_map = pp.load_ID_map(season)
fantrax = pd.read_csv('data/utils/'+str(season)+'/fantrax.csv')

In [342]:
fg_h,statcast_h,proj_sys_h = pp.load_raw_dataframes(position_group = 'hitters',season = 2021)
fg_p,statcast_p,proj_sys_p = pp.load_raw_dataframes(position_group = 'pitchers',season = 2021)

In [381]:

proj_mean_h = proj_sys_h.pivot_table(index=["season", "Name",'Team','playerid'], values=['PA','OBP','SLG','R','HR','RBI','NetSB'], aggfunc='mean').reset_index()
proj_mean_p = proj_sys_p.pivot_table(index=["season", "Name",'Team','playerid'], values=['IP','ERA'], aggfunc='mean').reset_index()


proj_metrics_h = proj_sys_h.pivot_table(index=["season", "Name",'Team','playerid'], columns='system', values=['PA','OBP','SLG','R','HR','RBI','NetSB']).reset_index()
col_list = []
for x in enumerate(proj_metrics_h.columns.tolist()):
    string  = x[1][0]+x[1][1]
    col_list.append(string)
proj_metrics_h.columns = col_list
proj_metrics_h

proj_metrics_p = proj_sys_p.pivot_table(index=["season", "Name",'Team','playerid'], columns='system', values=['IP','ERA']).reset_index()
col_list = []
for x in enumerate(proj_metrics_p.columns.tolist()):
    string  = x[1][0]+x[1][1]
    col_list.append(string)
proj_metrics_p.columns = col_list


proj_mean_h

,season,Name,Team,playerid,HR,NetSB,OBP,PA,R,RBI,SLG
0,2019.0,A.J. Kennedy,Padres,sa876029,0.00,0.00,0.219000,1.000000,0.000000,0.000000,0.263000
1,2019.0,A.J. Pollock,Dodgers,9256,20.50,12.25,0.324250,543.500000,75.750000,67.500000,0.459250
2,2019.0,A.J. Reed,Astros,16246,8.25,0.00,0.297750,176.500000,22.000000,26.000000,0.419000
3,2019.0,A.J. Simcox,Tigers,sa658901,0.00,0.00,0.270000,1.000000,0.000000,0.000000,0.328000
4,2019.0,AJ Graffanino,Braves,sa3008030,0.00,0.00,0.239000,1.000000,0.000000,0.000000,0.270000
...,...,...,...,...,...,...,...,...,...,...,...
6447,2021.0,Zach Reks,Dodgers,sa3004166,5.00,0.00,0.301667,158.333333,18.666667,20.333333,0.377333
6448,2021.0,Zachary Remillard,White Sox,sa659077,4.50,0.00,0.254500,243.000000,21.500000,19.000000,0.307000
6449,2021.0,Zachary Short,Tigers,sa873287,3.75,1.00,0.311000,150.250000,16.500000,13.750000,0.360000
6450,2021.0,Zachary Zehner,Yankees,sa829122,0.00,0.00,0.285000,1.000000,0.000000,0.000000,0.347000


In [233]:
master_h = pp.join_tables(fg_h,statcast_h, id_map, fantrax, position_group = 'hitters')
master_p = pp.join_tables(fg_p,statcast_p, id_map, fantrax, position_group = 'pitchers')

In [234]:
master_h1 = pp.preliminary_col_reduction(master_h,position_group = 'hitters')
master_p1 = pp.preliminary_col_reduction(master_p,position_group = 'pitchers')

## Training Data

In [235]:
career_h = pp.gen_career_stats(master_h1, position_group = 'hitters', pt_metric = pt_metric_h)
career_p = pp.gen_career_stats(master_p1, position_group = 'pitchers', pt_metric = pt_metric_p)

100%|██████████| 43/43 [00:03<00:00, 12.74it/s]


In [305]:
master_h2 = pp.feature_engineering(master_h1, career_h, position_group = 'hitters', pt_metric = pt_metric_h)
master_p2 = pp.feature_engineering(master_p1, career_p, position_group = 'pitchers', pt_metric = pt_metric_p)


100%|██████████| 105/105 [00:00<00:00, 114.25it/s][A

100%|██████████| 126/126 [00:00<00:00, 143.63it/s]


In [306]:
master_h3 = pp.drop_out_of_position(master_h2,position_group = 'hitters')
master_p3 = pp.drop_out_of_position(master_p2,position_group = 'pitchers')

In [307]:
master_h4 = pp.limit_years(master_h3,start_year=2015)
master_p4 = pp.limit_years(master_p3,start_year=2015)

In [308]:
master_sp4,master_rp4 = pp.split_pitchers(master_p4)

In [309]:
master_h5 = pp.make_targets(master_h4, counting_stats_h, per_metric_h)
master_sp5 = pp.make_targets(master_sp4, counting_stats_sp, per_metric_sp)
master_rp5 = pp.make_targets(master_rp4, counting_stats_rp, per_metric_rp)

In [310]:
master_h6 = pp.organize_training_columns(master_h5,position_group='hitters')
master_sp6 = pp.organize_training_columns(master_sp5,position_group='SP')
master_rp6 = pp.organize_training_columns(master_rp5,position_group='RP')

In [311]:
master_h7 = pp.remove_missing(master_h6,position_group='hitters')
master_sp7 = pp.remove_missing(master_sp6,position_group='SP')
master_rp7 = pp.remove_missing(master_rp6,position_group='RP')

In [312]:
master_h7.to_csv('data/processed/hitters/'+str(season)+'/modeling.csv')
master_sp7.to_csv('data/processed/SP/'+str(season)+'/modeling.csv')
master_rp7.to_csv('data/processed/RP/'+str(season)+'/modeling.csv')

## Scoring Data